A garbage scratchpad for scraping the contents of podcasts.joerogan.net before it gets taken down.

Want to scrape:
* audio file
* episode number
* publish date
* guest name
* guest description
* related links
* youtube slug

This stuff would only come from YouTube, maybe the API, maybe the pages themselves:
* length of youtube video (would help in calculating timestamp offset vs. mp3's)
* like/dislike count
* like/dislike ratio
* number of comments
* view count
* youtube timestamps, if present (sometimes in top comments)
* list of JRE clip excerpts


Important URL formats for the podcast site (from watching debugger->network->XHR)

curl http://podcasts.joerogan.net/podcasts/page/3?load

curl -s 'http://podcasts.joerogan.net/wp-admin/admin-ajax.php?action=loadPermalink&slug=bob-saget'



In [1]:
import lxml.html
import datetime
import requests
import os.path
import random
import glob
import json
import time

import pandas as pd

In [2]:
# Print full dataframes whenever I dump them.
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

In [3]:
directory = "indexhtml/" + datetime.datetime.now().strftime("%Y%m%d")
!mkdir -p {directory}

In [4]:
# Scrape the website index. 
# Just save off the HTML for now. Random sleep means
# we get ~6 req/min, so about 20 minutes total for the full thing.

# Hacky thing for not scraping the full thing on each update.
FULL = 170
UPDATE = 3
scrape = UPDATE

base = "http://podcasts.joerogan.net/podcasts/page/%d?load"

for x in range(1, scrape+1):
    try:
        r = requests.get(base % x)
        r.raise_for_status()
        fn = directory + "/%03d.html" % x
        with open(fn, 'w') as f:
            f.write(r.text)
            time.sleep(random.randint(2,5))
            
    except Exception:
        print(x)

In [5]:
fields = "type episode date guests description slug mp3 yt ytid".split()

# A series of fields, their xpaths, and functions to normalize them.
paths = {
    'episode': ('./ul[@class="podcast-icons"]//li/@data-episode-num[1]', lambda x:str(x[0])),
    'date' : ('./div[@class="podcast-date"]/h3/text()', lambda x: datetime.datetime.strptime(x[0], "%m.%d.%y").date()),
    'guests' : ('./div[@class="podcast-details"]/a[@class="ajax-permalink"]/h3/text()', lambda x: str(x[0])),
    'description' : ('./div[@class="podcast-details"]/div[@class="podcast-content"]//text()', lambda x: ''.join(x)),
    'slug' : ('(./div[@class="podcast-details"]/a[@class="ajax-permalink"])/@data-slug', lambda x: str(x[0])),
}


In [6]:
# Now extract the fields. Just put them into dicts for now; we'll chuck them in a dataframe later.
# Put those dicts into a dict, using the slug value as the key, so we can avoid duplicates when updating.

# Note that about 1-2% of the time, when you request any given page, their website returns page #1.
# It's very strange. Luckily, this method of updating solves that problem.

rows = dict()

# Aaaand go figure, around the time Miley Cyrus's episode was published, the managed to woefully
# break the HTML on page 1, nesting all the other episodes in the list under the first one.
# Well, for now, fuck it.
for fn in reversed(sorted(glob.glob("indexhtml/202008*/*.html"))):

    tree = lxml.html.parse(fn)
    episode_list = tree.find('//div[@id="podcasts"]//div[@class="main"]')
    episodes  = episode_list.xpath('./div[contains(@class, "episode")]')

    for ep in episodes:
        d = {k:None for k in fields}
        for f, (x,t) in paths.items():
            d[f] = t(ep.xpath(x))

        # Most episodes have a download link in these index pages.
        # However, the placement for these is kinda all over, so
        # we grab all the links and look for one that smells like
        # a match. The ones that don't have it, we can get from the
        # individual pages in another pass. (Just means more scraping.)

        links = ep.xpath('.//@href')
        mp3 = [l for l in links if 'libsyn.com' in l and d['episode'] in l]

        # If we find precisely one good link, save it.
        if len(mp3) == 1:
            d['mp3'] = mp3[0]

        rows[d['slug']] = d

In [7]:
df = pd.DataFrame(columns=fields, data=rows.values())

# Sorting this way helps things in the right order. Episodes increase with release date,
# though sometimes multiple come out on one day, and sometimes a single episode has pt1 and pt2.
df = df.sort_values(['date', 'episode', 'slug']).reset_index(drop=True)

In [8]:
# Turns out the show numbers aren't always numbers.
# There are a few oddballs, gaps, etc. and the MMA
# shows mess it up too.

# Try to convert every episode tag into a real number,
# possibly just a placeholder 0.
df['normalized'] = df.episode.str.extract('(\d*)')
df.normalized = '0' + df.normalized
df.normalized = df.normalized.astype(int)

df['standard'] = None

In [9]:
# Now we go through, and look for a monotonically increasing ep count.
# If it fits in the monotonic series, then we're counting it as part of the set of standard podcasts.

ep = 1
skipped = []

for i in range(len(df)):
    x = df.loc[i,'normalized']
    
    # If we skipped some numbers, save that.
    if x > ep: skipped.extend(range(ep, x))
    
    # If the current number is aligned with what you're loking for,
    # mark this podcast as a regular one. Otherwise, mark as not.
    y = x >= ep
    df.loc[i,'standard'] = y
    
    # If we found a regular podcast, start looking for the next on in the series.
    if y: ep = x + 1
        
        
print("skipped:", skipped)

skipped: [97, 108, 128, 172, 213, 677, 1036, 1093, 1117, 1366, 1423, 1440, 1477]


In [10]:
# Sanity check. Make sure the ones we got, plus the ones
# we skipped equals the number of regular podcasts
# released to date. Yup.

len(df[df.standard]) + len(skipped)

1527

In [11]:
# These seem to be missing entirely. Not on the website or YouTube: 97, 108, 128, 213
# These ones are only on YouTube: 1440.
# These are in the data, but the number was left blank: 1093, 1423, 1477
# 677 was labeled as 77.
# 172 was labeled as 173.
# 1036 was labeled as 1037.
# 1117 is also labeled as MMA show 27, even though there is another MMA show #27.
# 1366 was labeled as 1336

# I've seen other cases (eg. 1524) where it was initially only on YouTube, but added to
# the podcast site a few days later. I'm kinda surprised they don't do them at the same time.

# Also, this ones are clearly two part shows: 515, 701
# 706 also comes in two parts, and is titled as a fight companion(?), literally with a question mark
# so I'm putting that one in the regular list

# FC9 was labeled as a second 8.

# There's no FC14 on the website, but there is a July 11th, 2015 episode on YouTube. v=mgumau1KiRw

# There are two Fight Companions labeled as #35, throwing the later counts off by one.
# But shit, they seem to have really stuck to their guns on the bad numbering, so we'll do the same.

# FC38 was labeled as another 36, *after* 37 had been done. Wow.


# ---------- Other random shit
# 1423 (andrew-doyle) and 1477 (tony-hawk) don't have any download link on the website either.
# Doyle has the YouTube link, Hawk has neither, though his episode is on YouTube.


In [12]:
# The above corrections, plus some others.
# Set the unique slug for each podcast alongside the number it *should* have,
# and for the ones that are are not standard podcasts, also include the the type.

corrections = [
    ('podcast-172', 172),
    ('podcast-173-peter-joseph-brian-redban', 173),
    ('ari-shaffir-5', 515),
    ('josh-zepps-2', 677),
    ('honey-honey-part-2', 701),
    ('ari-shaffir-bert-kreischer-tom-segura-2', 1036),
    ('chris-kresser-2', 1037),
    ('owen-benjamin-kurt-metzger', 1093),
    ('jre-mma-show-27-with-tim-kennedy', 1117),
    ('richard-dawkins', 1366),
    ('andrew-doyle', 1423),
    ('tony-hawk', 1477),
    ('fight-companion-feb-14-2015', 9, "fc"),
    ('fight-companion-january-14-2018', 38, "fc"),
    ('brendan-schaub-fight-companion-part-2', 706),
]

for corr in corrections:
    slug, ep = corr[0:2]
    df.loc[df.slug==slug, 'episode'] = str(ep)
    if len(corr)==2:
        df.loc[df.slug==slug, 'type'] = 'podcast'
    else:
        df.loc[df.slug==slug, 'type'] = corr[2]
        

In [13]:
# Now start blocking these off into categories.
df.loc[df.standard, 'type'] = "podcast"

In [14]:
df.loc[df.type.isnull() & df.episode.str.match('JRQE'), 'type'] = 'jrqe'
df.loc[df.type.isnull() & df.episode.str.contains('FC'), 'type'] = 'fc'
df.loc[df.type.isnull() & df.slug.str.contains('companion'), 'type'] = 'fc'
df.loc[df.type.isnull() & df.guests.str.contains('MMA'), 'type'] = 'mma'
df.loc[df.type.isnull() & df.episode.str.match('\D'), 'type'] = 'misc'

In [15]:
# What's left?
df.loc[df.type.isnull()]

,type,episode,date,guests,description,slug,mp3,yt,ytid,normalized,standard
763,None,,2015-12-22,UFC Recap – Brendan Schaub & Eddie Bravo,"Joe discusses some of the fights from the UFConFOX fight card from December 19, 2015, and other topics, with Brendan Schaub & Eddie Bravo.",ufc-recap-brendan-schaub-eddie-bravo,http://traffic.libsyn.com/joeroganexp/p122115.mp3,None,None,0,False
1001,None,,2017-05-15,Podcast On A Plane UFC 211 Recap,"Joe sits down with Tony Hinchcliffe on a plane to discuss the weekend's events at UFC 211 in Dallas, TX.",podcast-on-a-plane-ufc-211-recap,None,None,None,0,False


In [16]:
# Now there are just two "UFC recap" shows popping up.
# They don't seem to be in any standard numbering, so into misc they go.
df.loc[df.type.isnull(), 'type'] = 'misc'

# Assign a number to all these motherfuckers. Wipe out the text based names.
misc = len(df[df.type=='misc'])
df.loc[df.type=="misc", "episode"] = list(range(1, misc+1))

In [17]:
# Strip all the text from the fight companion episodes
df.loc[df.type=="fc", 'episode'] = df.loc[df.type=="fc", 'episode'].str.extract('(\d+)').values

# Ditto the jrqe ones.
df.loc[df.type=="jrqe", 'episode'] = df.loc[df.type=="jrqe", 'episode'].str.extract('(\d+)').values

In [18]:
# Now turn the episodes into integers, index, and drop temp columns.
df.episode = df.episode.astype(int)

df.set_index(['type','episode'], inplace=True)
del df['normalized']
del df['standard']

In [19]:
# What do we have so far?
#df

In [20]:
!mkdir -p slugs

In [21]:
# Ah, fuck, it, I'll scrape every individual page, even if I already
# have the MP3 link. Why not? Might be useful later.

base = "http://podcasts.joerogan.net/wp-admin/admin-ajax.php?action=loadPermalink&slug=%s"

for x in df.slug.values:
    fn = "slugs/%s.html" % x
    
    # Since these slugs are unique, this is easier to make update without reptition.
    if os.path.exists(fn):
        continue
        
    try:
        r = requests.get(base % x)
        r.raise_for_status()
        with open(fn, 'w') as f:
            f.write(r.text)
            time.sleep(random.randint(2,5))
            
    except Exception:
        print(x)

In [22]:
paths = {
    'ytid' : ('.//a[@data-video-provider="youtube"]/@data-video-id', lambda x: x[0] if x else None),
    'mp3' : ('.//a[@class="download-episode"]/@href', lambda x: x[0] if x else None)
}

# change index for performance reasons here.
df = df.reset_index()
df = df.set_index('slug', drop=False)

for fn in glob.glob('slugs/*html'):

    slug = fn.partition('/')[2].replace('.html','')
    html = json.load(open(fn))['response']['html']
    tree = lxml.html.fromstring(html)
    
    d = {k:None for k in paths}
    for f, (x,t) in paths.items():
        d[f] = t(tree.xpath(x))

        # For everything we extract from the individual page, see if it lines up
        # with the index listings.
        fromindex = df.loc[slug,f]
        if fromindex:
            if fromindex != d[f]:
                print(df.loc[slug, ['type','episode']])
                print(fromindex, d[f])
                print()
                
        # In either case, save what we found on the individual pages.
        df.loc[slug, f] = d[f]

        
# Fix index up again.
df = df.set_index(['type','episode'])

# Looks like exactly one podcast had a broken "download this podcast" link.
# Whereas the link to play the podcast is correct. Not sure if that was a fatfingered typo
# or if the podcast was edited later for some reason and the link was only updated once.
# Anyway, we saved the correct one.

type       podcast
episode        468
Name: duncan-trussell-christopher-ryan-2, dtype: object
http://traffic.libsyn.com/joeroganexp/p468.mp3 http://traffic.libsyn.com/joeroganexp/p468a.mp3



In [23]:
# Turns out there's a bunch with broken YouTube links - not just old stuff (that was on Vimeo) either,
# but new stuff as well.
print(len(df[df.ytid.isnull()]))
#df[df.ytid.isnull()]

165


In [24]:
# Coming up with two where the link to play the MP3 on the site is busted. Seemingly missing.
# Luckily, they're both on YouTube, even if Tony Hawk's YT link isn't on the podcast site either.
# Especially since Andrew Doyle is a funny motherfucker.
df[~df.mp3.astype(bool)]

date        guests  \
type    episode                             
podcast 1423     2020-02-05  Andrew Doyle   
        1477     2020-05-20     Tony Hawk   

                                                                                                                                                                                                                                                                   description  \
type    episode                                                                                                                                                                                                                                                                  
podcast 1423     #1423. Andrew Doyle is a British comedian, playwright, journalist, political satirist and is creator of the fictitious character Titania McGrath. The new book "Woke: A Guide to Social Justice" by Titania McGrath is now available: https://amzn.to/36X2GoG   
        1477                                                                                                                                              #1477. Tony Hawk is a professional skateboarder, actor, stuntman, and the owner of the skateboard company Birdhouse.   

                         slug mp3    yt         ytid  
type    episode                                       
podcast 1423     andrew-doyle      None  NIxhH85cQMY  
        1477        tony-hawk      None         None

In [25]:
# Save this shit for later.
df.to_csv('rogan-scrape.csv')

In [26]:

doubles = [515, 701, 706] # Handle these manually, because software engineering is digital garbage collection


i=0
f = open('dl-jre.sh','w')
for (t, ep), row in df.iterrows():
    fn = "%s%04d.mp3" % (t, ep)
    
    if ep in doubles or not row.mp3:
        continue
    
    if not os.path.exists('/media/sshfs/jre/podcast/%s' % fn):
        i+=1
        print('wget "%s" -O %s' %(row.mp3, fn), file=f)
        
f.close()
!mv dl-jre.sh /media/sshfs/jre/podcast

In [27]:
# Downloading these manually to get the in the right spots.
df.loc[(slice(None),doubles), :]

date                                       guests  \
type    episode                                                            
podcast 515      2014-06-27                                  Ari Shaffir   
        515      2014-06-27                                  Ari Shaffir   
        701      2015-09-28                         Honey Honey (Part 1)   
        701      2015-09-28                         Honey Honey (Part 2)   
        706      2015-10-09  Brendan Schaub & Fight Companion ? (Part 1)   
        706      2015-10-09  Brendan Schaub & Fight Companion ? (Part 2)   

                                                                                                                                                                                                                                                                              description  \
type    episode                                                                                                                                                                                                                                                                             
podcast 515                                                                                                                                       #515. (Part 1) Ari Shaffir is a stand-up comedian and also hosts his own podcasts "Ari Shaffir's Skeptic Tank" and "Punch Drunk Sports"   
        515                                                                                                                                       #515. (Part 2) Ari Shaffir is a stand-up comedian and also hosts his own podcasts "Ari Shaffir's Skeptic Tank" and "Punch Drunk Sports"   
        701                                                     #701. Honey Honey is a band, featuring members Suzanne Santo and Ben Jaffe, from Los Angeles, CA. They released a new album this summer called "3" and are currently touring all over -- http://honeyhoneyband.com/events   
        701                                                     #701. Honey Honey is a band, featuring members Suzanne Santo and Ben Jaffe, from Los Angeles, CA. They released a new album this summer called "3" and are currently touring all over -- http://honeyhoneyband.com/events   
        706      #706. Brendan Schaub is a mixed martial artist and also a former college & pro football player. He also hosts a podcast with Bryan Callen called "The Fighter & The Kid" available on iTunes.\r\n\r\nJoe & Brendan also watch fights that take place on October 9, 2015.   
        706      #706. Brendan Schaub is a mixed martial artist and also a former college & pro football player. He also hosts a podcast with Bryan Callen called "The Fighter & The Kid" available on iTunes.\r\n\r\nJoe & Brendan also watch fights that take place on October 9, 2015.   

                                                  slug  \
type    episode                                          
podcast 515                              ari-shaffir-4   
        515                              ari-shaffir-5   
        701                         honey-honey-part-1   
        701                         honey-honey-part-2   
        706      brendan-schaub-fight-companion-part-1   
        706      brendan-schaub-fight-companion-part-2   

                                                             mp3    yt  \
type    episode                                                          
podcast 515       http://traffic.libsyn.com/joeroganexp/p515.mp3  None   
        515      http://traffic.libsyn.com/joeroganexp/p515a.mp3  None   
        701       http://traffic.libsyn.com/joeroganexp/p701.mp3  None   
        701      http://traffic.libsyn.com/joeroganexp/p701b.mp3  None   
        706      http://traffic.libsyn.com/joeroganexp/p706a.mp3  None   
        706      http://traffic.libsyn.com/joeroganexp/p706b.mp3  None   

                        ytid  
type    episode               
podcast 515  

In [28]:
# mma0039.mp3:       empty
# mma0043.mp3:       empty
# mma0094.mp3:       empty

# These ones all return 404's. However, they're all on YouTube.
# Strangely, I can see the original video ID for mma43 is now private,
# but it was reposted by the official channel as OYSk66YiFwQ

In [29]:
df.loc[('mma',[39,43,94]),:]

date                                         guests  \
type episode                                                              
mma  39       2018-08-29  JRE MMA Show #39 with Donald “Cowboy” Cerrone   
     43       2018-10-02           JRE MMA Show #43 with Brendan Schaub   
     94       2020-04-10           JRE MMA Show #94 with Brendan Schaub   

                                                                                        description  \
type episode                                                                                          
mma  39                               #039. Joe sits down with UFC fighter Donald "Cowboy" Cerrone.   
     43       #43. Joe is joined by Brendan Schaub to preview UFC #229 and some more future fights.   
     94              #094. Joe sits down with with Brendan Schaub to discuss the "upcoming" fights.   

                                                    slug  \
type episode                                               
mma  39       jre-mma-show-39-with-donald-cowboy-cerrone   
     43              jre-mma-show-43-with-brendan-schaub   
     94              jre-mma-show-94-with-brendan-schaub   

                                                               mp3    yt  \
type episode                                                               
mma  39       http://traffic.libsyn.com/joeroganexp/mmashow39a.mp3  None   
     43       http://traffic.libsyn.com/joeroganexp/mmashow43a.mp3  None   
     94        http://traffic.libsyn.com/joeroganexp/mmashow94.mp3  None   

                     ytid  
type episode               
mma  39       Yvi7y10yNyg  
     43       BoVXS7e0NRw  
     94       cF7_AaY9tCg

In [30]:
df

date  \
type    episode               
podcast 1        2009-12-24   
        2        2009-12-29   
        3        2010-01-06   
        4        2010-01-13   
        5        2010-01-21   
        6        2010-01-28   
        7        2010-02-03   
        8        2010-02-10   
        9        2010-02-24   
        10       2010-03-03   
        11       2010-03-09   
        12       2010-03-20   
        13       2010-03-31   
        14       2010-04-03   
        15       2010-04-07   
        16       2010-04-13   
        17       2010-04-21   
        18       2010-04-27   
        19       2010-05-04   
        20       2010-05-12   
        21       2010-05-18   
        22       2010-05-25   
        23       2010-06-02   
        24       2010-06-08   
        25       2010-06-15   
        26       2010-06-16   
        27       2010-06-24   
        28       2010-06-29   
        29       2010-07-06   
        30       2010-07-14   
        31       2010-07-20   
        32       2010-07-28   
        33       2010-08-03   
        34       2010-08-10   
        35       2010-08-11   
        36       2010-08-21   
        37       2010-08-24   
        38       2010-08-31   
        39       2010-09-02   
        40       2010-09-07   
        41       2010-09-13   
        42       2010-09-20   
        43       2010-09-23   
        44       2010-09-28   
        45       2010-09-29   
        46       2010-10-05   
        47       2010-10-09   
        48       2010-10-12   
        49       2010-10-19   
        50       2010-10-21   
        51       2010-10-26   
        52       2010-10-28   
        53       2010-11-02   
        54       2010-11-09   
        55       2010-11-16   
        56       2010-11-23   
        57       2010-11-26   
        58       2010-11-29   
        59       2010-12-01   
        60       2010-12-07   
        61       2010-12-08   
        62       2010-12-15   
        63       2010-12-17   
        64       2010-12-21   
        65       2010-12-21   
        66       2010-12-28   
        67       2010-12-30   
        68       2011-01-04   
        69       2011-01-05   
        70       2011-01-11   
        71       2011-01-13   
        72       2011-01-18   
        73       2011-01-19   
        74       2011-01-25   
        75       2011-01-26   
        76       2011-02-01   
        77       2011-02-03   
        78       2011-02-08   
        79       2011-02-10   
        80       2011-02-15   
        81       2011-02-16   
        82       2011-02-21   
        83       2011-02-22   
        84       2011-02-28   
        85       2011-03-01   
        86       2011-03-04   
        87       2011-03-08   
        88       2011-03-10   
        89       2011-03-15   
        90       2011-03-16   
        91       2011-03-22   
        92       2011-03-22   
        93       2011-03-28   
        94       2011-03-31   
        95       2011-04-05   
        96       2011-04-08   
        98       2011-04-13   
        99       2011-04-18   
        100      2011-04-19   
        101      2011-04-26   
        102      2011-04-27   
        103      2011-05-02   
        104      2011-05-04   
        105      2011-05-09   
        106      2011-05-10   
        107      2011-05-12   
        109      2011-05-31   
        110      2011-06-01   
        111      2011-06-06   
        112      2011-06-07   
        113      2011-06-13   
        114      2011-06-14   
        115      2011-06-20   
        116      2011-06-21   
        117      2011-06-28   
        118      2011-06-29   
        119      2011-07-05   
        120      2011-07-06   
        121      2011-07-11   
        122      2011-07-19   
        123      2011-07-20   
        124      2011-07-24   
        125      2011-07-27   
        126      2011-07-31   
        127      2011-08-03   
        129      2011-08-11   
        130      2011-08-16   
        131      2011-08

In [ ]:
# Reading it back in later.
import pandas as pd
df = pd.read_csv('./rogan-scrape.csv')